In [2]:
#notebook to scrape community votes from transfermarkt.de

#import necessary packages
import pandas as pd
import requests, bs4

In [3]:
#function to get all links for a season
def get_links(url):
    headers = {"User-Agent":"Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = bs4.BeautifulSoup(response.text, 'lxml')
    table = soup.find_all("tr", {"class": "no-border tm-user-tendenz"})
    table_teams = soup.find_all("tr", {"class": "table-grosse-schrift"})
    probs = dict()
    for i in range(0,10):
        table_tmp = table[i].find_all('td')
        table_teams_tmp =  table_teams[i].find_all('td')
        if 'home_team' in probs:
            probs ['home_team'].append (table_teams_tmp[0].contents[3].contents[0])
        else:
            probs ['home_team'] = [table_teams_tmp[0].contents[3].contents[0]]
        if 'away_team' in probs:
            probs ['away_team'].append (table_teams_tmp[7].contents[1].contents[0])
        else:
            probs ['away_team'] = [table_teams_tmp[7].contents[1].contents[0]]
        if 'home_prob' in probs:
            probs ['home_prob'].append (table_tmp[0].contents[1].contents[0].contents[0])
        else:
            probs ['home_prob'] = [table_tmp[0].contents[1].contents[0].contents[0]]
        if 'draw_prob' in probs:
            probs ['draw_prob'].append (table_tmp[1].contents[1].contents[0].contents[0])
        else:
            probs ['draw_prob'] = [table_tmp[1].contents[1].contents[0].contents[0]]
        if 'away_prob' in probs:
            probs ['away_prob'].append (table_tmp[2].contents[1].contents[0].contents[0])
        else:
            probs ['away_prob'] = [table_tmp[2].contents[1].contents[0].contents[0]]

    out = pd.DataFrame.from_dict(probs)

    return  out

test = get_links('https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2020&spieltag=4')
#get_links('https://fbref.com/en/comps/9/2020-2021/schedule/2020-2021-Premier-League-Scores-and-Fixtures')
test

,home_team,away_team,home_prob,draw_prob,away_prob
0,Chelsea,Crystal Palace,93.9 %,4.1 %,2.0 %
1,Everton,Brighton,95.5 %,4.5 %,%
2,Leeds,Man City,6.3 %,13.7 %,80.0 %
3,Newcastle,Burnley,84.0 %,12.1 %,3.9 %
4,Leicester,West Ham,97.1 %,2.4 %,0.5 %
5,Southampton,West Brom,65.7 %,26.6 %,7.7 %
6,Arsenal,Sheff Utd,97.1 %,2.4 %,0.5 %
7,Wolves,Fulham,84.6 %,10.6 %,4.8 %
8,Man Utd,Spurs,38.5 %,43.8 %,17.8 %
9,Aston Villa,Liverpool,1.4 %,1.9 %,96.6 %


In [20]:
#basic link for each season
string_19 = 'https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2019&spieltag='
string_20 = 'https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2020&spieltag='
string_21 = 'https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2021&spieltag='
df_19 = get_links('https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2019&spieltag=4')
df_20 = get_links('https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2020&spieltag=4')
df_21 = get_links('https://www.transfermarkt.com/premier-league/spieltag/wettbewerb/GB1/plus/?saison_id=2021&spieltag=4')

#get community votes from gameweek 4
def get_communtity_votes(stri, df):
    for i in range (5, 39):
        stri_tmp = stri
        word ="{}".format(i)
        stri_tmp += word
        df_tmp = get_links(stri_tmp)
        df = pd.concat([df, df_tmp])
    return df
votes_19 = get_communtity_votes(string_19, df_19)
votes_20 = get_communtity_votes(string_20, df_20)
votes_21 = get_communtity_votes(string_21, df_21)

In [28]:
#tranform votes to %
def transform_votes(df):
    df.replace(['Leeds','Leicester','Man City' ,'Man Utd','Newcastle', 'Norwich','Sheff Utd', 'Spurs'], ['Leeds United', 'Leicester City', 'Manchester City', 'Manchester Utd', 'Newcastle Utd', 'Norwich City','Sheffield Utd', 'Tottenham'], inplace=True)
    df['home_prob'] = pd.to_numeric(df['home_prob'].str.slice(start=-7, stop=-1),errors='coerce')/100
    df['draw_prob'] = pd.to_numeric(df['draw_prob'].str.slice(start=-7, stop=-1),errors='coerce')/100
    df['away_prob'] = pd.to_numeric(df['away_prob'].str.slice(start=-7, stop=-1),errors='coerce')/100
    df = df.fillna(0)
    df['game_id'] = df['home_team'] + df['away_team']
    return df
transfermarkt_probs_19 = transform_votes(votes_19)
transfermarkt_probs_20 = transform_votes(votes_20)
transfermarkt_probs_21 = transform_votes(votes_21)

In [32]:
#rename columns
transfermarkt_probs_19 = transfermarkt_probs_19.rename(columns={'home_prob':'H','draw_prob':'D','away_prob':'A'})
transfermarkt_probs_20 = transfermarkt_probs_20.rename(columns={'home_prob':'H','draw_prob':'D','away_prob':'A'})
transfermarkt_probs_21 = transfermarkt_probs_21.rename(columns={'home_prob':'H','draw_prob':'D','away_prob':'A'})